<a href="https://colab.research.google.com/github/srigouri/CE888-lab/blob/main/Labs/lab6/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
np.set_printoptions(precision = 3)
import pandas as pd
import operator
import matplotlib.pyplot as plt
from IPython.display import Image

In [2]:
ratings = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', header=None )
print(ratings)

       0      1      2      3      4     5      6     7     8      9    ...  \
0       74  -7.82   8.79  -9.66  -8.16 -7.52  -8.50 -9.85  4.17  -8.98  ...   
1      100   4.08  -0.29   6.36   4.37 -2.38  -9.66 -0.73 -5.34   8.88  ...   
2       49  99.00  99.00  99.00  99.00  9.03   9.27  9.03  9.27  99.00  ...   
3       48  99.00   8.35  99.00  99.00  1.80   8.16 -2.82  6.21  99.00  ...   
4       91   8.50   4.61  -4.17  -5.39  1.36   1.60  7.04  4.61  -0.44  ...   
...    ...    ...    ...    ...    ...   ...    ...   ...   ...    ...  ...   
24978  100   0.44   7.43   9.08   2.33  3.20   6.75 -8.79 -0.53  -8.74  ...   
24979   91   9.13  -8.16   8.59   9.08  0.87  -8.93 -3.50  5.78  -8.11  ...   
24980   39  99.00  99.00  99.00  99.00 -7.77  99.00  6.70 -6.75  99.00  ...   
24981   37  99.00  99.00  99.00  99.00 -9.71  99.00  4.56 -8.30  99.00  ...   
24982   72   2.43   2.67  -3.98   4.27 -2.28   7.33  2.33  4.56   6.75  ...   

         91     92     93     94     95     96     

In [3]:
ratings.drop(columns=[0],axis=1, inplace=True)
ratings = ratings.replace(99.00, np.NaN)
ratings.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,...,2.82,NaN,NaN,NaN,NaN,NaN,-5.63,NaN,NaN,NaN
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,NaN,NaN,NaN,NaN,9.03,9.27,9.03,9.27,NaN,NaN,...,NaN,NaN,NaN,9.08,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,8.35,NaN,NaN,1.80,8.16,-2.82,6.21,NaN,1.84,...,NaN,NaN,NaN,0.53,NaN,NaN,NaN,NaN,NaN,NaN
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [4]:
mean_rating = ratings.mean(axis=0)
(pd.DataFrame(mean_rating)).nlargest(10,0)
ratings.iloc[5621, 17]

arr = ratings.values
print(arr.shape)
rated = np.where(~np.isnan(arr))
print(len(rated[0]), rated[1].shape)

(24983, 100)
1810455 (1810455,)


In [5]:
def replace(orig, percentage=0.1):
  ndata = orig.copy()
  rated = np.where(~np.isnan(arr))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  ndata[rated[0][idx], rated[1][idx]] = 99
  return ndata, (rated[0][idx], rated[1][idx])

n_arr, idx = replace(arr, 0.1)
print(arr[idx[0][0], idx[1][0]])
n_arr[idx[0][0], idx[1][0]]

2.38


99.0

In [6]:
n_latent_factors = 2
latent_u_preferences = np.random.random((arr.shape[0], n_latent_factors))
latent_i_features = np.random.random((arr.shape[1], n_latent_factors))

def predict(u_id, i_id):
    u_preference = latent_u_preferences[u_id]
    i_preference = latent_i_features[i_id]
    return u_preference.dot(i_preference)

def train(u_id, i_id, rating, alpha=0.0001):
    predicted_r = predict(u_id, i_id)
    error =  predicted_r - rating
    upref_values = latent_u_preferences[u_id]
    latent_u_preferences[u_id] -= alpha * error * latent_i_features[i_id]
    latent_i_features[i_id] -= alpha * error * upref_values
    return error

def sgd(iterations):
    mse_history = []
    for iter in range(iterations):
        error = []
        for u_id in range(latent_u_preferences.shape[0]):
            for i_id in range(latent_i_features.shape[0]):
                rating = arr[u_id, i_id]
                if (~np.isnan(rating)) and rating!=99 :
                    err = train(u_id, i_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iter % 10) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iter, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [7]:
predictions = latent_u_preferences.dot(latent_i_features.T)
predictions
values = [zip(arr[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = ratings.columns
comparison_data

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 0.8579499625083877)","(8.79, 1.4206570024660052)","(-9.66, 0.9849806526800896)","(-8.16, 0.98071638709053)","(-7.52, 0.6476656404513055)","(-8.5, 0.6825568453403278)","(-9.85, 0.8732889847076831)","(4.17, 0.8298637172823892)","(-8.98, 0.054564733220274)","(-4.76, 1.3680042080496488)",...,"(2.82, 0.5361089355347429)","(nan, 0.8475322530977975)","(nan, 0.7362175970334994)","(nan, 0.8520892446100241)","(nan, 1.3084769853166784)","(nan, 0.6313568203570119)","(-5.63, 0.7872640232964769)","(nan, 1.1927353715879443)","(nan, 0.7576460305513404)","(nan, 1.309262500015852)"
1,"(4.08, 0.49033713967041914)","(-0.29, 1.0586088070787063)","(6.36, 0.6262195826471981)","(4.37, 0.6938074186075137)","(-2.38, 0.4060665549099319)","(-9.66, 0.42445961059072673)","(-0.73, 0.5484538927502977)","(-5.34, 0.7234504868464217)","(8.88, 0.051251333675209426)","(9.22, 0.9993303105359619)",...,"(2.82, 0.2953122349870411)","(-4.95, 0.7903017291294822)","(-0.29, 0.5245693753264176)","(7.86, 0.562566022718742)","(-0.19, 0.9615374447022332)","(-2.14, 0.3439632836967354)","(3.06, 0.6663790515830029)","(0.34, 0.8905024586865763)","(-4.32, 0.6215151968904712)","(1.07, 0.9736081773753823)"
2,"(nan, 0.47413935096834064)","(nan, 0.6117830395290857)","(nan, 0.49987506591471326)","(nan, 0.4483132395349934)","(9.03, 0.3326930311710421)","(9.27, 0.35306315457261844)","(9.03, 0.44793891260941854)","(9.27, 0.2835339965761398)","(nan, 0.01605449614552474)","(nan, 0.6031935270419639)",...,"(nan, 0.3040661516110182)","(nan, 0.2534189123650315)","(nan, 0.3339231398030687)","(9.08, 0.41779295023528296)","(nan, 0.5729466150631343)","(nan, 0.36076930071637897)","(nan, 0.282986692258126)","(nan, 0.5124163040213021)","(nan, 0.28624900768579276)","(nan, 0.5652143088044798)"
3,"(nan, 0.8509807647790821)","(8.35, 0.9169659384711362)","(nan, 0.8507225385304613)","(nan, 0.7067818526026759)","(1.8, 0.5707553820237001)","(8.16, 0.608451731553729)","(-2.82, 0.7677336610474079)","(6.21, 0.32599789071567975)","(nan, 0.014085807684533673)","(1.84, 0.9229721624859146)",...,"(nan, 0.5538718930405891)","(nan, 0.23029095926249166)","(nan, 0.523129676908809)","(0.53, 0.6943765592187797)","(nan, 0.8714537486655454)","(nan, 0.6598884716503807)","(nan, 0.34903544030949085)","(nan, 0.7664009216722526)","(nan, 0.3753959007087259)","(nan, 0.8490455332949786)"
4,"(8.5, 0.4190385759357395)","(4.61, 0.8374027964684181)","(-4.17, 0.5179034943475432)","(-5.39, 0.5565651809481954)","(1.36, 0.337227152756651)","(1.6, 0.3533679300833409)","(7.04, 0.45524512164388653)","(4.61, 0.5502990003117315)","(-0.44, 0.03832617247442473)","(5.73, 0.794703995763964)",...,"(5.19, 0.2553951892594314)","(5.58, 0.5919508730265673)","(4.27, 0.41998175948428346)","(5.19, 0.46015183589530795)","(5.73, 0.7634351833475594)","(1.55, 0.29854986071612977)","(3.11, 0.5104514191488468)","(6.55, 0.7040618245628283)","(1.8, 0.4797304143509329)","(1.6, 0.7705810522402992)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,"(0.44, 0.5433083190839874)","(7.43, 1.2641596822437409)","(9.08, 0.7172639630261782)","(2.33, 0.8180399829417639)","(3.2, 0.46320965132600267)","(6.75, 0.48301743893799454)","(-8.79, 0.6259470859949663)","(-0.53, 0.8937157546281511)","(-8.74, 0.06420611998763187)","(7.23, 1.1876878090268823)",...,"(8.83, 0.3231167043431126)","(-1.21, 0.9887693671869714)","(9.22, 0.6196130346124588)","(-6.7, 0.6512797469854306)","(8.45, 1.1444178407214622)","(9.03, 0.37488487335804904)","(6.55, 0.818381265485112)","(8.69, 1.0639026450534113)","(8.79, 0.7583432541140818)","(7.43, 1.1620888936099798)"
24979,"(9.13, 0.13326230457243898)","(-8.16, 0.7363314601242233)","(8.59, 0.28528311392570194)","(9.08, 0.4310072022037224)","(0.87, 0.17567220234560207)","(-8.93, 0.17785558833477838)","(-3.5, 0.23881086895728082)","(5.78, 0.6497771220875751)","(-8.11, 0.05042393877098076)","(4.9, 0.6671394869513031)",...,"(-1.17, 0.06009728544656243

In [8]:
num_iter = 100
hist = sgd(num_iter)
plt.figure()
plt.plot(np.arange(0, num_iter, 10), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

Iteration 0/100:	MSE=26.180997


KeyboardInterrupt: ignored

**Movie**

In [9]:
df1 = pd.ExcelFile('https://github.com/albanda/CE888/blob/master/lab4-recommender/movies_latent_factors.xlsx?raw=true')
df_i = pd.read_excel('https://github.com/albanda/CE888/blob/master/lab4-recommender/movies_latent_factors.xlsx?raw=true',sheet_name='Items',index_col=None)
df_u = pd.read_excel('https://github.com/albanda/CE888/blob/master/lab4-recommender/movies_latent_factors.xlsx?raw=true',sheet_name='Users', index_col=None)

In [10]:
df_i.head()

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
1,12,Finding Nemo (2003),-0.342185,-0.296586,-0.385962,2.443297,-1.097015,-0.619465,0.572887,0.329516,-0.712228,0.323554,-0.561948,-0.142405,-0.564415,0.506876,1.274993
2,13,Forrest Gump (1994),-2.240888,-0.438815,-2.275177,0.614548,0.904469,0.711919,-0.420876,1.302036,-0.868418,-0.160122,0.633667,0.133138,-0.330276,-2.209004,-0.419092
3,14,American Beauty (1999),-0.634531,2.186059,-0.066681,0.086197,0.517558,-0.185319,-0.412352,0.063841,0.075937,-0.577682,0.526803,-1.465557,-0.819682,0.549010,-0.681191
4,22,Pirates of the Caribbean: The Curse of the Bla...,0.517348,-1.456763,0.369161,0.073903,-0.098332,-0.594722,0.828888,0.033626,-0.707414,0.717877,1.394326,-0.786986,1.429191,0.855746,-2.292566


In [29]:
num_factors = df_u.shape[1]-1
u_pref = df_u[df_u['User'] == 3519].iloc[:, -num_factors:].values[0]
i_feat = df_i.iloc[:, -num_factors:].values
pref = i_feat.dot(u_pref)
pref

array([ 0.485, -0.082,  0.02 ,  0.14 , -0.177,  0.113,  0.084,  0.067,
        0.093,  0.258, -0.045,  0.178,  0.034,  0.017,  0.046,  0.021,
        0.04 ,  0.121,  0.062,  0.08 ,  0.116,  0.041, -0.065, -0.006,
       -0.086,  0.199, -0.036,  0.198,  0.072,  0.073,  0.061,  0.159,
        0.21 ,  0.113,  0.087,  0.016,  0.063, -0.128,  0.12 , -0.03 ,
        0.011,  0.035, -0.008, -0.019,  0.119,  0.027, -0.095,  0.008,
        0.156, -0.148,  0.034, -0.334, -0.178,  0.15 , -0.058,  0.041,
       -0.083, -0.06 , -0.071,  0.246,  0.028, -0.11 ,  0.067, -0.185,
        0.024, -0.222, -0.261, -0.042, -0.512,  0.07 , -0.051, -0.008,
       -0.087,  0.014, -0.039, -0.204, -0.141,  0.503,  0.449, -0.022,
        0.058, -0.116, -0.017,  0.005, -0.016, -0.533, -0.069,  0.074,
       -0.041, -0.076, -0.545, -0.058,  0.015, -0.097, -0.079, -0.073,
       -0.017, -0.074, -0.057, -0.061])

In [30]:
df_fact = df_i[['Movie ID','Factor6']]
df_fact.sort_values('Factor6')

,Movie ID,Factor6
58,672,-2.237460
57,671,-2.232775
43,558,-1.507203
36,393,-1.269396
84,2502,-1.225494
...,...,...
77,1891,2.306469
78,1892,2.495382
0,11,2.513260
85,3049,2.592889


In [34]:
top_val = pd.DataFrame(pref).nlargest(3, 0)
top_indc = top_val.index
list()

TypeError: ignored

In [32]:
df_fact = df_i[['Movie ID','Factor1']]

val = df_fact[df_fact['Movie ID']==2501]['Factor1'].values[0]

round(val,3)

-0.831

In [18]:
df_fact = df_i[['Movie ID','Factor7']]

val = df_fact[df_fact['Movie ID']==120]['Factor7'].values[0]

round(val,3)

1.134

In [33]:
ratings.iloc[2501,3519]

IndexError: ignored

In [23]:
df_i.iloc[list(top_indc)]

,Movie ID,Title,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,Factor11,Factor12,Factor13,Factor14,Factor15
77,1891,Star Wars: Episode V - The Empire Strikes Back...,-1.732354,-1.031041,2.221631,0.310886,-0.900830,2.306469,-1.847214,0.231435,-0.096332,-0.673135,-1.010806,0.657123,0.141075,0.998841,-0.115583
0,11,Star Wars: Episode IV - A New Hope (1977),-1.521848,-1.038507,2.027269,0.247933,-0.594548,2.513260,-1.848910,0.476710,-0.224146,-0.760681,-0.973915,0.862379,0.403861,1.129616,-0.248806
78,1892,Star Wars: Episode VI - Return of the Jedi (1983),-1.353408,-0.965083,2.099767,0.143335,-0.538405,2.495382,-1.925854,0.264722,-0.467617,-0.553613,-0.912548,0.737907,0.302659,0.969395,-0.102475
